---

Extrair noticias dos sites de verificação de fake news: 
<br> https://lupa.uol.com.br/jornalismo/categoria/verifica%C3%A7%C3%A3o 
<br> https://g1.globo.com/fato-ou-fake/
<br> https://apublica.org/tipo_nota/entrelinhas-do-poder/
<br> https://www.e-farsas.com/

---

In [32]:

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [33]:
'''
Cria uma instância do objeto que controla o início e parada do ChromeDriver
'''
s = Service('./chromedriver.exe')
'''
Define as opções do webdriver do Chrome
'''
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")


#### Carregando a página e scrollando para obter todos os links

In [34]:
link = "https://lupa.uol.com.br/jornalismo/categoria/verifica%C3%A7%C3%A3o "
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options) # abre o browser Chrome
driver.get(link)
sleep(8)

total_iterations = 3000

for counter in tqdm(range(total_iterations), desc="Scrolling and Clicking"):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(8)
        button_class = "sc-cwSeag jxHWSb"
        button_xpath = f"//button[@class='{button_class}']"
        button = WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, button_xpath)))
        button.click()
        sleep(8)
    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Scrolling and Clicking:  14%|█▎        | 410/3000 [55:05<5:48:01,  8.06s/it]


KeyboardInterrupt: 

#### Coletando as informações e os links das notícias

In [35]:

elements = driver.find_elements(by=By.XPATH, value = "//div[@id='init']//div[@class='sc-dkrFOg cbDCWR']")
data = []
for element in elements:
    link = element.find_element(By.XPATH, "*//a[@class='sc-eDWCr hNENvd']").get_attribute('href')
    author = element.find_element(By.XPATH, ".//div[@class='sc-hLBbgP gREmsY']//span[@class='sc-eDvSVe hQCdpY']").text
    title = element.find_element(By.XPATH, ".//div[@class='sc-hLBbgP eRlKZk']//span").text
    headline = element.find_element(By.XPATH, ".//div[@class='sc-hLBbgP gQITEQ']//span/p").text
    data.append({'Author': author, 'Link': link, 'Title': title, 'Headline': headline})

In [36]:
len(data)

480

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

,Author,Link,Title,Headline
0,Por Ítalo Rômany,https://lupa.uol.com.br/jornalismo/2023/12/29/...,Formigas com foto de Cristo e dança da picanha...,Suzane von Richthofen foi convidada para integ...
1,Por Maiquel Rosauro,https://lupa.uol.com.br/jornalismo/2023/12/27/...,Lula e Bolsonaro são tema das verificações mai...,Levantamento da Lupa mostra as dez verificaçõe...
2,Por Evelyn Fagundes,https://lupa.uol.com.br/jornalismo/2023/12/21/...,Filha de Ana Maria Braga fez afirmações falsas...,Circula nas redes um vídeo em que Mariana Maff...
3,Por Maiquel Rosauro,https://lupa.uol.com.br/jornalismo/2023/12/21/...,Decoração natalina na Praça da Sé não é inicia...,"Post alega que o governador de São Paulo, Tarc..."
4,Por Gabriela Soares,https://lupa.uol.com.br/jornalismo/2023/12/20/...,É golpe programa ‘Abono Natalino’ com benefíci...,Link compartilhado nas redes sociais promove u...


In [ ]:
df.iloc[0]

Author                                       Por Ítalo Rômany
Link        https://lupa.uol.com.br/jornalismo/2023/12/29/...
Title       Formigas com foto de Cristo e dança da picanha...
Headline    Suzane von Richthofen foi convidada para integ...
Name: 0, dtype: object

In [ ]:
df.to_csv("links_lupa_uol.csv", index=False)

In [ ]:
df = pd.read_csv("links_lupa_uol.csv")

In [ ]:
df.insert(loc=0, column='RawText', value='')

In [ ]:
#Keep runing on the links not extracted yet
#df = pd.read_csv("noticias_lupa.csv")

In [ ]:
#df = df[df.RawText.isna() == True]

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)

In [ ]:
for i in tqdm(range(len(df)), desc="Access the page and extracting text"):
    try:
        link = df.iloc[i]['Link']

        driver.get(link)
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"*//div[@id='root']")))
        
        elements = driver.find_elements(by=By.XPATH, value = "*//div[@id='root']")
        
        text_str = elements[0].text
        
        #Retira ruido antes de "Entrar" 
        text_str = text_str.split('Entrar')[1]
        
        #Retira ruido após "Leia Também"
        text_str = text_str.split('Leia também')[0]
        
        df.at[i, 'RawText'] = text_str
        
        if i % 7 == 0:
            sleep(1)
            df.to_csv("noticias_lupa.csv", index = False)
    except:
        print(f"Problem on {i}")
        print(link)
driver.quit()

Access the page and extracting text: 100%|██████████| 480/480 [08:28<00:00,  1.06s/it]


In [ ]:
df.head()

,RawText,Author,Link,Title,Headline
0,\nRetrospectiva\nFormigas com foto de Cristo e...,Por Ítalo Rômany,https://lupa.uol.com.br/jornalismo/2023/12/29/...,Formigas com foto de Cristo e dança da picanha...,Suzane von Richthofen foi convidada para integ...
1,\n‌\n\n‌\n\n‌\n\n‌\n\n‌\n\n‌,Por Maiquel Rosauro,https://lupa.uol.com.br/jornalismo/2023/12/27/...,Lula e Bolsonaro são tema das verificações mai...,Levantamento da Lupa mostra as dez verificaçõe...
2,\n‌\n\n‌\n\n‌\n\n‌\n\n‌\n\n‌\n\nSobre a lupa\n...,Por Evelyn Fagundes,https://lupa.uol.com.br/jornalismo/2023/12/21/...,Filha de Ana Maria Braga fez afirmações falsas...,Circula nas redes um vídeo em que Mariana Maff...
3,\n‌\n\n‌\n\n‌\n\n‌\n\n‌\n\n‌,Por Maiquel Rosauro,https://lupa.uol.com.br/jornalismo/2023/12/21/...,Decoração natalina na Praça da Sé não é inicia...,"Post alega que o governador de São Paulo, Tarc..."
4,\n‌\n\n‌\n\n‌\n\n‌\n\n‌\n\n‌\n\nSobre a lupa\n...,Por Gabriela Soares,https://lupa.uol.com.br/jornalismo/2023/12/20/...,É golpe programa ‘Abono Natalino’ com benefíci...,Link compartilhado nas redes sociais promove u...


In [ ]:
df.to_csv("noticias_lupa.csv", index = False)